In [ ]:
import pandas as pd

# Load the MovieLens dataset
movies_url = "https://raw.githubusercontent.com/sundeepblue/movie_rating_prediction/master/movie_metadata.csv"
movies = pd.read_csv(movies_url)
movies.head()


,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [ ]:
!pip install numpy pandas scikit-learn transformers


In [ ]:
# Select relevant features
movies = movies[['movie_title', 'genres', 'director_name', 'actor_1_name', 'actor_2_name', 'imdb_score']]
movies.dropna(inplace=True)

# Clean movie titles
movies['movie_title'] = movies['movie_title'].apply(lambda x: x.strip())


<ipython-input-3-80cea67c699d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies.dropna(inplace=True)
<ipython-input-3-80cea67c699d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['movie_title'] = movies['movie_title'].apply(lambda x: x.strip())


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Convert genres to TF-IDF features
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies['genres'])

# Compute cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Function to recommend movies
def recommend_movies(title, cosine_sim=cosine_sim):
    idx = movies[movies['movie_title'] == title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    movie_indices = [i[0] for i in sim_scores]
    return movies['movie_title'].iloc[movie_indices]

# Test the recommender
recommend_movies("Avatar")


,movie_title
15,Man of Steel
39,The Amazing Spider-Man 2
236,Star Wars: Episode III - Revenge of the Sith
237,Star Wars: Episode II - Attack of the Clones
240,Star Wars: Episode I - The Phantom Menace


In [ ]:
from transformers import pipeline

# Explicitly specify the model name
qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")

# Define the context for the RAG system (movie data)
context = " ".join(movies['movie_title'].values)

# Function for question answering
def answer_question(question):
    result = qa_pipeline(question=question, context=context)
    return result['answer']

# Test the system
answer_question("What is the name of a popular movie?")


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

"Fool's Gold Funny People The Kingdom Talladega Nights"

In [ ]:
answer_question("Recommend some movies similar to Avatar.")

'Characters Absentia'

In [ ]:
!pip install transformers sentence-transformers faiss-cpu streamlit -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.4 MB/s eta 0:00:00


In [ ]:
!pip install yfinance
import yfinance as yf

# Download historical data for a financial asset (for example, Tesla stock)
tsla_data = yf.download('TSLA', start='2020-01-01', end='2023-01-01')
tsla_data.head()


[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,28.299999,28.713333,28.114000,28.684000,28.684000,142981500
2020-01-03,29.366667,30.266666,29.128000,29.534000,29.534000,266677500
2020-01-06,29.364668,30.104000,29.333332,30.102667,30.102667,151995000
2020-01-07,30.760000,31.441999,30.224001,31.270666,31.270666,268231500
2020-01-08,31.580000,33.232666,31.215334,32.809334,32.809334,467164500


In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from sentence_transformers import SentenceTransformer, util
import faiss
import numpy as np
import yfinance as yf
import pandas as pd

# Load data (you can replace this with any professional financial dataset)
# Example: Stock data for different companies
tickers = ['TSLA', 'AAPL', 'GOOGL', 'AMZN', 'MSFT']
documents = []

for ticker in tickers:
    data = yf.download(ticker, start='2020-01-01', end='2023-01-01')
    summary = f"Summary of {ticker}: Starting price: {data['Close'][0]}, Ending price: {data['Close'][-1]}, Highest price: {data['High'].max()}, Lowest price: {data['Low'].min()}"
    documents.append(summary)

# Initialize models
retriever_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
document_embeddings = retriever_model.encode(documents, convert_to_tensor=True)

# Initialize FAISS for retrieval
dimension = document_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(document_embeddings))

# GPT-2 for text generation
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
generator_model = GPT2LMHeadModel.from_pretrained('gpt2')

def retrieve(query):
    """Retrieve the most relevant document based on user query."""
    query_embedding = retriever_model.encode(query, convert_to_tensor=True)
    _, indices = index.search(np.array([query_embedding]), k=1)
    return documents[indices[0][0]]

def generate_response(prompt):
    """Generate a response using the GPT-2 model."""
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    outputs = generator_model.generate(inputs, max_length=150, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


[*********************100%***********************]  1 of 1 completed
<ipython-input-3-80f55c75a4a7>:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  summary = f"Summary of {ticker}: Starting price: {data['Close'][0]}, Ending price: {data['Close'][-1]}, Highest price: {data['High'].max()}, Lowest price: {data['Low'].min()}"
[*********************100%***********************]  1 of 1 completed
<ipython-input-3-80f55c75a4a7>:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  summary = f"Summary of {ticker}: Starting price: {data['Close'][0]}, Ending price: {data['Close'][-1]}, Highest price: {data['High'].max()}, Lowest 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
import os
import yfinance as yf
import numpy as np
import faiss
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from sentence_transformers import SentenceTransformer


In [ ]:
# Store Hugging Face access token securely
os.environ['HUGGINGFACE_TOKEN'] = 'your_hugging_face_key'
huggingface_token = os.getenv('HUGGINGFACE_TOKEN')


In [ ]:
# Download stock data for major companies
tickers = ['TSLA', 'AAPL', 'GOOGL', 'AMZN', 'MSFT']
documents = []

for ticker in tickers:
    data = yf.download(ticker, start='2020-01-01', end='2023-01-01')
    summary = f"Summary of {ticker}: Starting price: {data['Close'][0]}, Ending price: {data['Close'][-1]}, Highest price: {data['High'].max()}, Lowest price: {data['Low'].min()}"
    documents.append(summary)

# Print a sample summary
print(documents[0])


[*********************100%***********************]  1 of 1 completed
<ipython-input-5-e0cbc08228d1>:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  summary = f"Summary of {ticker}: Starting price: {data['Close'][0]}, Ending price: {data['Close'][-1]}, Highest price: {data['High'].max()}, Lowest price: {data['Low'].min()}"
[*********************100%***********************]  1 of 1 completed
<ipython-input-5-e0cbc08228d1>:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  summary = f"Summary of {ticker}: Starting price: {data['Close'][0]}, Ending price: {data['Close'][-1]}, Highest price: {data['High'].max()}, Lowest pr

Summary of TSLA: Starting price: 28.68400001525879, Ending price: 123.18000030517578, Highest price: 414.4966735839844, Lowest price: 23.367332458496094



<ipython-input-5-e0cbc08228d1>:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  summary = f"Summary of {ticker}: Starting price: {data['Close'][0]}, Ending price: {data['Close'][-1]}, Highest price: {data['High'].max()}, Lowest price: {data['Low'].min()}"


In [ ]:
# Initialize SentenceTransformer model for retrieval
retriever_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
document_embeddings = retriever_model.encode(documents, convert_to_tensor=True)

# Initialize FAISS index for fast retrieval
dimension = document_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(document_embeddings))


In [ ]:
# Initialize GPT-2 model and tokenizer using Hugging Face token
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', use_auth_token=huggingface_token)
generator_model = GPT2LMHeadModel.from_pretrained('gpt2', use_auth_token=huggingface_token)


In [ ]:
!pip install openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.1/374.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00


In [ ]:
import openai

# Set your OpenAI API key
openai.api_key = 'openai_api_key'

In [ ]:
def retrieve(query):
    """Retrieve the most relevant financial document based on the query."""
    query_embedding = retriever_model.encode(query, convert_to_tensor=True)
    _, indices = index.search(np.array([query_embedding]), k=1)
    return documents[indices[0][0]]

def generate_response_openai(prompt):
    """Generate a response using GPT-3.5 or GPT-4."""
    response = openai.Completion.create(
        engine="gpt-3.5-turbo",  # Use "gpt-4" if you have access
        prompt=prompt,
        max_tokens=150,
        temperature=0.7,        # Adjust as needed
        top_p=0.9,              # Nucleus sampling
        frequency_penalty=0.5,  # Reduces repetition
        presence_penalty=0.5    # Encourages more diverse responses
    )
    return response.choices[0].text.strip()


In [ ]:
def chatbot_openai(query):
    """Main function to process the query, retrieve relevant data, and generate a response using OpenAI API."""
    # Step 1: Retrieve relevant financial data
    retrieved_doc = retrieve(query)

    # Step 2: Structure the prompt clearly for GPT-3.5 or GPT-4
    prompt = (
        f"User: {query}\n"
        f"Financial Insight: {retrieved_doc}\n"
        f"Assistant: Based on the financial data and your question about cryptocurrency diversification, here's some tailored advice:"
    )

    # Step 3: Generate a response using GPT-3.5 or GPT-4
    response = generate_response_openai(prompt)

    return response


In [ ]:
# Test with an example query
user_query = "How do I diversify my cryptocurrency portfolio?"
response = chatbot_openai(user_query)

print(f"User: {user_query}")
print(f"Assistant: {response}")


NameError: name 'retriever_model' is not defined

In [ ]:
!pip install openai==0.28

In [ ]:
user_query = "What are the best stocks to invest in for the next 5 years?"
response = chatbot(user_query)

print(f"User: {user_query}")
print(f"Assistant: {response}")


User: What are the best stocks to invest in for the next 5 years?
Assistant: User: What are the best stocks to invest in for the next 5 years?
Financial Insight: Summary of AMZN: Starting price: 94.90049743652344, Ending price: 84.0, Highest price: 188.6540069580078, Lowest price: 81.30149841308594
Assistant: Based on the financial data and your question about cryptocurrency diversification, here's some tailored advice:
1. Invest in a diversified portfolio.
2. Invest in a diversified portfolio.
3. Invest in a diversified portfolio.
4. Invest in a diversified portfolio.
5. Invest in a diversified portfolio.
6. Invest in a diversified portfolio.
7. Invest in a diversified portfolio.
8. Invest in a diversified portfolio.
9. Invest in a diversified portfolio.
10. Invest in a diversified portfolio.
11. Invest in a diversified portfolio.


In [ ]:
user_query = "How should I manage my finances as a new graduate?"
response = chatbot(user_query)

print(f"User: {user_query}")
print(f"Assistant: {response}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


User: How should I manage my finances as a new graduate?
Assistant: User: How should I manage my finances as a new graduate?
Financial Insight: Summary of MSFT: Starting price: 160.6199951171875, Ending price: 239.82000732421875, Highest price: 349.6700134277344, Lowest price: 132.52000427246094
Assistant: How should I manage my finances as a new graduate?
Financial Insight: Summary of MSFT: Starting price: 160.6199951171875, Ending price: 239.82000732421875, Highest price: 349.6700134277344, Lowest price: 132.52000427246094
Assistant: How should I manage my finances


In [ ]:
# Test with an example query
user_query = "How should I manage my finances as a new graduate?"
response = chatbot(user_query)

print(f"User: {user_query}")
print(f"Assistant: {response}")


User: How should I manage my finances as a new graduate?
Assistant: User: How should I manage my finances as a new graduate?
Financial Insight: Summary of MSFT: Starting price: 160.6199951171875, Ending price: 239.82000732421875, Highest price: 349.6700134277344, Lowest price: 132.52000427246094
Assistant: Based on the financial data and your question, here's some financial advice specifically for you:
1. You should be able to make a profit on your MSFT.
2. You should be able to make a profit on your MSFT. 3. You should be able to make a profit on your MSFT. 4. You should be able to make a profit on your


In [ ]:
user_query = "What are the best stocks to invest in for the next 5 years?"
response = chatbot(user_query)

print(f"User: {user_query}")
print(f"Assistant: {response}")


User: What are the best stocks to invest in for the next 5 years?
Assistant: User: What are the best stocks to invest in for the next 5 years?
Financial Insight: Summary of AMZN: Starting price: 94.90049743652344, Ending price: 84.0, Highest price: 188.6540069580078, Lowest price: 81.30149841308594
Assistant: Based on the financial data and your question, here's some financial advice specifically for you:
1. Invest in stocks that are not in the top 10% of the market.
2. Invest in stocks that are in the top 10% of the market.
3. Invest in stocks that are in the top 10% of the market.
4. Invest in stocks that are in


In [ ]:
# Test with an example query
user_query = "How do I diversify my cryptocurrency portfolio?"
response = chatbot(user_query)

print(f"User: {user_query}")
print(f"Assistant: {response}")


User: How do I diversify my cryptocurrency portfolio?
Assistant: User: How do I diversify my cryptocurrency portfolio?
Financial Insight: Summary of AMZN: Starting price: 94.90049743652344, Ending price: 84.0, Highest price: 188.6540069580078, Lowest price: 81.30149841308594
Assistant: Based on the financial data and your question about cryptocurrency diversification, here's some tailored advice:
1. Invest in a diversified portfolio.
2. Invest in a diversified portfolio with a high level of diversification.
3. Invest in a diversified portfolio with a high level of diversification.
4. Invest in a diversified portfolio with a high level of diversification.
5. Invest in a diversified portfolio with a high level of diversification.
6. Invest in a diversified portfolio with a high level of diversification.
7. Invest in a diversified portfolio with a high level of diversification.
8.
